In [39]:
from dash import Dash, dash_table, dcc, html
from dash.dependencies import Input, Output
import pandas as pd
from flask import Flask
from Librerias_SAP import SAP_GUI, Funtions
import pandas as pd
import re
from collections import OrderedDict
import dash
from dash import dash_table as dt
from dash import dcc
from dash import html
from dash.dependencies import Input
from dash.dependencies import Output
import numpy as np
import dash_bootstrap_components as dbc
from datetime import datetime, timedelta
import plotly.graph_objects as go

In [40]:
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password

Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][2]
password=Keys["Password"][2]
# Initialize session
session=SAP_GUI.SessionSAP(user,password)

In [41]:
# Disposiciones ZSD110
Name_ZSD110_FIRME="Pendiente_Firme"  # Revisar suele no guardar
Name_ZSD110_MERCADEO="Pendiente_Mercadeo"  # Revisar suele no guardar
Name_ZSD110_TOTAL="Pendiente_Total"  # Revisar suele no guardar

default_column_Canal01_Pendientes=['Can.distr.', 'Denomin.', 'Seccion', 'Estado', 'Solic.', 'Nombre',
       'Pedido', 'Clase Pedi', 'Creado el', 'Pedido.1', 'Posición', 'Material',
       'Vlr.Neto P', 'Moneda', 'Cantidad P', 'Cantidad C', 'UM venta',
       'Vlr.Neto C', 'Status Glo', 'Status Tot', 'Status Ent', 'Status Ent.1',
       'Status Ent.2']  

default_column_Canal01_Entregado=['Can.distr.', 'Denomin.', 'Seccion', 'Estado', 'Solic.', 'Nombre',
       'Entrega', 'Posicion E', 'F.Creacion', 'Pedido', 'Clase Pedi',
       'F.Creacion.1', 'Pedido.1', 'Posicion P', 'Material', 'Moneda',
       'Cantidad E', 'UM venta', 'Valor Neto', 'StTotPick.', 'StatGlWM',
       'StTotMovMe', 'Stat.fact.', 'Status PT']

default_column_Pedidos=['OrgVt', 'CDis', 'BqEn', 'Solic.', 'PrimFeEntr', 'ÚltEntrega',
       'FePrefEnt.', 'ClVt', 'Valor neto', 'Mon.', 'Doc.venta', 'ST',
       'Solicitante', 'Nº pedido cliente', 'Descrip.breve', 'Nº de cliente 1',
       'Contador de pedidos', 'Dirección', 'Barrio', 'Población', 'Teléfono']

Columns_Data_ZSD10_Exito_Cencosub_ZVMI=['Fecha doc.', 'Solicitante', 'Cliente', 'Nº de cliente 1', 'Marca',
       'Material', 'Número de material', 'Unidades', 'UM', 'ValorNeto',
       'Nº pedido cliente', 'Doc.venta', 'ClVt', 'Solic.', 'OrgVt', 'CDis',
       'OfVta', 'GVen', 'Organización de ventas', 'Mon.', 'Jquía.productos',
       'Total Pedidos', 'Plataforma', 'Fe.PedCpas', 'Creado por']

Columns_Data_ZSD038_Exito_Cencosub_Rechazados=['Doc.comer.', 'Fecha doc.', 'Destinat.', 'Nombre 1', 'Cód.Vend.',
       'Nombre vendedor', 'MR', 'Denominación', 'Material', 'Denominación.1',
       'Marca', 'Categoría', 'Ctd.confirmada', 'Valor neto', 'Mon.', 'CDis',
       'Solic.', 'Nombre']

Columns_Data_ZSD10=['Fecha doc.', 'Solicitante', 'Cliente', 'Nº de cliente 1', 'Marca',
       'Material', 'Número de material', 'Unidades', 'UM', 'ValorNeto',
       'Nº pedido cliente', 'Doc.venta', 'ClVt', 'Solic.', 'OrgVt', 'CDis',
       'OfVta', 'GVen', 'Organización de ventas', 'Mon.', 'Jquía.productos',
       'Total Pedidos', 'Plataforma', 'Fe.PedCpas', 'Creado por']

Column_Data_ZSD76=['Solic.', 'Solicitante', 'Fecha salida mercancía', 'Material',
       'Número de material', 'Cantidad entrega', 'UM', 'MARCA', 'CATEGORIA',
       'Entrega', 'ClEnt', 'Clase de entrega', 'ClVt',
       'Clase de documento de ventas', 'OrgVt', 'CDis', 'Doc.venta',
       'Prc.neto', 'Mon.', 'Destinat.', 'Destinatario de mercancías',
       'Nº pedido cliente', 'Fecha creación Pedido', 'Cont.']

"""----------------------------------------------LEER ARCHIVOS--------------------------------"""
#Data Pendientes
Canal01_Pendiente=pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\Canal01_PEN.txt",skiprows=1,delimiter="\t")
Canal01_Pendiente=Funtions.Clean_Columns(Canal01_Pendiente)
Canal01_Pendiente=Funtions.default_column(default_column_Canal01_Pendientes,Canal01_Pendiente)

#Dta Entregados
Canal01_Entregado=pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\Canal01_ENT.txt",skiprows=1,delimiter="\t")
Canal01_Entregado=Funtions.Clean_Columns(Canal01_Entregado)
Canal01_Entregado=Funtions.default_column(default_column_Canal01_Entregado,Canal01_Entregado)

In [42]:
"""""-------------------------------------------Limpio los datos-----------------------------------"""

# Esta session combierte los valores monetarios en enteros str a int para posteriores calculos
Canal01_Pendiente['Vlr.Neto P'], Canal01_Pendiente['Cantidad P']=Funtions.Clean_Num_List(Canal01_Pendiente['Vlr.Neto C'], Canal01_Pendiente['Cantidad P'])
Canal01_Entregado['Valor Neto'], Canal01_Entregado['Cantidad E']=Funtions.Clean_Num_List(Canal01_Entregado['Valor Neto'],Canal01_Entregado['Cantidad E'])

# Esta session combierte colunmas de float .0 a enteros int, tambien recibe valores Null
Canal01_Pendiente['Pedido.1'],Canal01_Entregado['Pedido.1']=Funtions.Estandarizo_Pedidos(Canal01_Pendiente['Pedido.1'],Canal01_Entregado['Pedido.1'])


# ¿Preguntar cantidad pendiente P y C significado

In [43]:
"""Seccion para coompletar los valores necesarios para trabajar"""

Agenda=["85","20","146","149","50","138","45"]
# En este caso solo aplica para Entrega y pendiente en el cual completa con 00 los pedidos del Exito
Canal01_Entregado['Pedido.1']=Funtions.complete_pedidos(Canal01_Entregado['Pedido.1'],Agenda)
Canal01_Pendiente['Pedido.1']=Funtions.complete_pedidos(Canal01_Pendiente['Pedido.1'],Agenda)

In [44]:
"""---------------------------------------Unifico los dataframes de pedido y entregas--------------------------------------------------------------"""
Default_Column_Informe=['Canal','Denomin.','Solic.','Nombre','Pedido','Clase Pedido','Material','Valor Neto','Cantidad']

longuitud_Pendiente=len(Canal01_Pendiente)
longuitud_Entrega=len(Canal01_Entregado)


Canal01_Pendiente[['Can.distr.','Denomin.','Solic.','Nombre','Pedido.1','Clase Pedi','Material','Vlr.Neto P','Cantidad P']]
Canal01_Entregado[['Can.distr.', 'Denomin.','Solic.','Nombre','Pedido.1','Clase Pedi','Material','Valor Neto','Cantidad E']]


Canal01_Pendiente_Informe=Canal01_Pendiente[['Can.distr.','Denomin.','Solic.','Nombre','Pedido.1','Clase Pedi','Material','Vlr.Neto P','Cantidad P']]
Canal01_Entregado_Informe=Canal01_Entregado[['Can.distr.', 'Denomin.','Solic.','Nombre','Pedido.1','Clase Pedi','Material','Valor Neto','Cantidad E']]


print(longuitud_Pendiente,longuitud_Entrega)


"""---En esta sección analizar si borro duplicados-----"""

Canal01_Entregado_Pendiente_Informe=pd.concat([Funtions.default_column(Default_Column_Informe,Canal01_Pendiente_Informe),Funtions.default_column(Default_Column_Informe,Canal01_Entregado_Informe)]).reset_index(drop=True).drop_duplicates()

3896 8938


In [45]:
""""------------------------------------ Estandarizo datos de columnas del informe de las lineas que deberian ser numericos------------------------------------------------"""

try:
    Canal01_Entregado_Pendiente_Informe['Valor Neto'],Canal01_Entregado_Pendiente_Informe['Cantidad']=Funtions.Clean_Num_List(Canal01_Entregado_Pendiente_Informe['Valor Neto'],Canal01_Entregado_Pendiente_Informe['Cantidad'])
except:
    print("intento")
    pass


In [46]:
"""------------------------Optener datos de Agenda_Exito y Cencosub------------------"""
#Agenda Exito y Cencosub
Agenda_Exito=["0085","0020","0146","0149","0050","0138","0045"]
Agenda_Cencosub=["93","122","127","95","60"]


"""-----------------------------Aplico filtro en Agenda exito Entrega----------------------------------"""
Filtro_Agenda_Exito_Entrega_Pendiente=Funtions.Search_Agenda_Exito(Canal01_Entregado_Pendiente_Informe['Pedido'],Agenda_Exito)
Filtro_Agenda_Cencosub_Entrega_Pendiente=Funtions.Search_agenda_Cencosub(Canal01_Entregado_Pendiente_Informe["Pedido"],Agenda_Cencosub)
Filtro_Sin_Agenda_Cencosub_Entrega_Pendiente=Filtro_Agenda_Exito_Entrega_Pendiente+Filtro_Agenda_Cencosub_Entrega_Pendiente

In [47]:
"""-------------------------------------------------Aplico filtro para el consolidados------------------------------------------------------------"""


"""------------------------------Canal 01 Entrega con agenda Exito-----------------------"""
#Agenda Exito
Filtro_Canal01_Entregado_Pendiente_Exito=(Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Agenda_Exito_Entrega_Pendiente))   #Excluyo con ~
Canal01_Entregado_Pendiente_Agenda_Exito=Canal01_Entregado_Pendiente_Informe.loc[Filtro_Canal01_Entregado_Pendiente_Exito].reset_index(drop=True)
Canal01_Entregado_Pendiente_Sin_Agenda_Exito=Canal01_Entregado_Pendiente_Informe.loc[~Filtro_Canal01_Entregado_Pendiente_Exito].reset_index(drop=True)

#Agenda Censosud
"""------------------------------Canal 01 Entrega con agenda Cencosub-----------------------"""
Filtro_Canal01_Entregado_Pendiente_Cencosub=(Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Agenda_Cencosub_Entrega_Pendiente))   #Excluyo con ~
Canal01_Entregado_Pendiente_Agenda_Cencosub=Canal01_Entregado_Pendiente_Informe.loc[Filtro_Canal01_Entregado_Pendiente_Cencosub].reset_index(drop=True)
Canal01_Entregado_Pendiente_Sin_Agenda_Cencosub=Canal01_Entregado_Pendiente_Informe.loc[~Filtro_Canal01_Entregado_Pendiente_Cencosub].reset_index(drop=True)


In [48]:
"""------------------------------Canal 01 Entrega con y sin agenda Exito y Cencosub-----------------------"""
Filtro_Canal01_Entregado_Pendiente_Sin_Exito_Cencosub=(Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Sin_Agenda_Cencosub_Entrega_Pendiente)) 
Canal01_Entregado_Pendiente_Sin_Agenda_Exito_Cencosub=Canal01_Entregado_Pendiente_Informe.loc[~Filtro_Canal01_Entregado_Pendiente_Sin_Exito_Cencosub].reset_index(drop=True)

In [49]:

"""----------------------------------------------Creao tabla dinamica general--------------------------------------------------------------------------------"""
Tabla_dinamica_Canal01_Entregado_Pendiente_Informe=Canal01_Entregado_Pendiente_Informe.pivot_table(index=['Pedido','Clase Pedido'],aggfunc={'Material':'count','Cantidad':sum,'Valor Neto':sum}).reset_index()

In [50]:
# Tabla dinácamica Exito
Table_dinamica_Exito_entrega_pendiente=Tabla_dinamica_Canal01_Entregado_Pendiente_Informe[Tabla_dinamica_Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Agenda_Exito_Entrega_Pendiente)]
# Tabla dinácamica Exito Cencosud
Table_dinamica_Cencosub_entrega_pendiente=Tabla_dinamica_Canal01_Entregado_Pendiente_Informe[Tabla_dinamica_Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Agenda_Cencosub_Entrega_Pendiente)]
# Tabla dinácamica sin agenda Exito y Cencosud
Table_dinamica_Sin_Exito_Cencosub_entrega_pendiente=Tabla_dinamica_Canal01_Entregado_Pendiente_Informe[~Tabla_dinamica_Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Sin_Agenda_Cencosub_Entrega_Pendiente)]


In [51]:
Table_dinamica_Cencosub_entrega_pendiente

,Pedido,Clase Pedido,Cantidad,Material,Valor Neto
73,122-443114,ZVMI,1260.0,533,14630289.0
74,127-447239,ZVMI,4546.0,910,61435131.0
90,60-1165930,ZEDI,12.0,1,148440.0
95,93-488974,ZVMI,1679.0,525,20728610.0
96,93-489284,ZEDI,18.0,2,276138.0
97,95-497213,ZVMI,4135.0,697,56235892.0
98,95-497519,ZEDI,6.0,1,107538.0


# Preguntar con los valores Null

In [52]:
pedidos_Exito_Cencosub=pd.concat([Table_dinamica_Exito_entrega_pendiente['Pedido'],Table_dinamica_Cencosub_entrega_pendiente['Pedido']])

#Busco en la tansaccion ZSD037 los pedidos de las tablas dinamicas tanto de Exito y Cencosub

Transsaccion_ZSD037="zsd037"
SAP_GUI.Search_Pedidos_ZSD037(Transsaccion_ZSD037,pedidos_Exito_Cencosub,session)

In [53]:
Name_ZSD037="Pedidos pendientes Exito y Cencosub"
Ruta_ZSD037_Exito_Cencosub="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_ZSD037,session,Ruta_ZSD037_Exito_Cencosub)

In [54]:
# Utilizo la transaccion para buscar ZSD037 en entrega para las fechas
url_ZSD037_Pedidos_Exito_Cencosub="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_ZSD037)+".txt"

Data_ZSD037_Pedidos_Exito_Cencosub=pd.read_csv(url_ZSD037_Pedidos_Exito_Cencosub,skiprows=5,delimiter="\t")
Data_ZSD037_Pedidos_Exito_Cencosub=Funtions.Clean_Columns(Data_ZSD037_Pedidos_Exito_Cencosub)
Data_ZSD037_Pedidos_Exito_Cencosub=Funtions.default_column(default_column_Pedidos,Data_ZSD037_Pedidos_Exito_Cencosub)
Data_ZSD037_Pedidos_Exito_Cencosub=Data_ZSD037_Pedidos_Exito_Cencosub[Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente'].notnull()]


# Ver metodo estandarizar pedidos y ver las fechas de los que no pertenecen a la agendas Exito y Cencosub

In [55]:
# Puede que se represente inconsistencia entre str y int
try:
    Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente']=Funtions.Estandarizo_Pedidos(Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente'])
except:
    pass

In [56]:
#Dataframe de interes
Data_ZSD037_Pedidos_Pendientes_Fecha=Data_ZSD037_Pedidos_Exito_Cencosub[['Nº pedido cliente','PrimFeEntr','ÚltEntrega']].drop_duplicates()

In [57]:
Data_ZSD037_Pedidos_Pendientes_Fecha

,Nº pedido cliente,PrimFeEntr,ÚltEntrega
0,0020665185,06.07.2023,15.07.2023
156,0020666496,07.07.2023,15.07.2023
204,0020666438,08.07.2023,08.07.2023
486,0020666497,08.07.2023,08.07.2023
825,0020666501,08.07.2023,08.07.2023
...,...,...,...
8021,60-1165930,10.07.2023,11.07.2023
8030,93-488974,13.07.2023,13.07.2023
8066,93-489284,07.07.2023,22.07.2023
8084,95-497213,13.07.2023,13.07.2023


In [58]:
Table_dinamica_Exito_entrega_pendiente

,Pedido,Clase Pedido,Cantidad,Material,Valor Neto
0,0020665185,ZVMI,305.0,85,6707501.0
1,0020666438,ZED1,91.0,4,4675568.0
2,0020666496,ZVMI,565.0,166,6167618.0
3,0020666497,ZVMI,54.0,36,965294.0
4,0020666501,ZVMI,28.0,24,239525.0
...,...,...,...,...,...
67,0149102983,ZEDI,192.0,22,2577024.0
68,0149102984,ZEDI,24.0,4,531702.0
69,0149102999,ZEDI,24.0,2,258144.0
70,0149103000,ZEDI,36.0,11,689724.0


In [59]:
#---------------------------------Tabla dinamica Exito---------------------------------------------
Tabla_Agenda_Exito=pd.merge(Data_ZSD037_Pedidos_Pendientes_Fecha,Table_dinamica_Exito_entrega_pendiente,how="right",left_on="Nº pedido cliente",right_on="Pedido")[['Nº pedido cliente', 'PrimFeEntr', 'ÚltEntrega',
       'Clase Pedido', 'Cantidad', 'Material', 'Valor Neto']]

Agenda_name_Exito={"0085":"FUNZA","0020":"VEGAS","0146":"BARRANQUILLA","0149":"BUCARAMANGA","0050":"CALI","0138":"PEREIRA","0045":"SURTIMAYORISTAS"}

Tabla_Agenda_Exito["Plataforma"]=Tabla_Agenda_Exito["Nº pedido cliente"].apply(lambda x: Agenda_name_Exito[re.findall("("+"|".join(list(Agenda_name_Exito.keys()))+")",x)[0]] if re.findall("("+"|".join(list(Agenda_name_Exito.keys()))+")",x)!=[] else x)


In [60]:
#Dataframe de interes

Tabla_Agenda_Cencusub=pd.merge(Data_ZSD037_Pedidos_Pendientes_Fecha,Table_dinamica_Cencosub_entrega_pendiente,how="right",left_on="Nº pedido cliente",right_on="Pedido")[['Nº pedido cliente', 'PrimFeEntr', 'ÚltEntrega',
       'Clase Pedido', 'Cantidad', 'Material', 'Valor Neto']]

Agenda_name_Cencosub={"93-":"MEDELLIN","122-":"BARRANQUILLA","127-":"BUCARAMANGA","95-":"CALI","60-":"BOGOTA"}

Tabla_Agenda_Cencusub["Plataforma"]=Tabla_Agenda_Cencusub["Nº pedido cliente"].apply(lambda x: Agenda_name_Cencosub[re.findall("\d*-",x)[0]] if re.findall("\d*-",x)[0] in list(Agenda_name_Cencosub.keys()) else None)

# Pedido Firme Mercadeo Total

In [61]:
# ZSD110 FIRME
url_ZSD110_FIRME="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_ZSD110_FIRME)+".txt"
Data_ZSD110_FIRME=pd.read_csv(url_ZSD110_FIRME,skiprows=1,delimiter="\t")
Data_ZSD110_FIRME=Funtions.Clean_Columns(Data_ZSD110_FIRME)
Data_ZSD110_FIRME["Lineas"]=Funtions.Clean_Num_List(Data_ZSD110_FIRME["Lineas"])


# ZSD110 MERCADE0
url_ZSD110_MERCADEO="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_ZSD110_MERCADEO)+".txt"
Data_ZSD110_MERCADEO=pd.read_csv(url_ZSD110_MERCADEO,skiprows=1,delimiter="\t")
Data_ZSD110_MERCADEO=Funtions.Clean_Columns(Data_ZSD110_MERCADEO)
Data_ZSD110_MERCADEO["Lineas"]=Funtions.Clean_Num_List(Data_ZSD110_MERCADEO["Lineas"])


# ZSD110 Pendiente Total
url_ZSD110_PENDIENTES="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_ZSD110_TOTAL)+".txt"
Data_ZSD110_PENDIENTES=pd.read_csv(url_ZSD110_PENDIENTES,skiprows=1,delimiter="\t")
Data_ZSD110_PENDIENTES=Funtions.Clean_Columns(Data_ZSD110_PENDIENTES)
Data_ZSD110_PENDIENTES["Lineas"]=Funtions.Clean_Num_List(Data_ZSD110_PENDIENTES["Lineas"])

"""---------------------CAMBIAR INDICES------------------------------------------"""
Channels_Name={1:"MODERNO",2:"TRADICIONAL",7:"TIENDAS DE BELLEZA",8:"PROFESIONAL",12:"INSTITUCIONAL",14:"HARD DISCOUNTERS",15:"ECOMMERCE",19:"ALTERNATIVO",50:"ALMACENES INTERNOS"}

#Aplicar transformacion
Data_ZSD110_MERCADEO["Can.distr."]=Data_ZSD110_MERCADEO["Can.distr."].replace(Channels_Name)
Data_ZSD110_FIRME["Can.distr."]=Data_ZSD110_FIRME["Can.distr."].replace(Channels_Name)
Data_ZSD110_PENDIENTES["Can.distr."]=Data_ZSD110_PENDIENTES["Can.distr."].replace(Channels_Name)

In [62]:
#Estandarizo Datos
# Cambio a valores Numericos Mercadeo
Data_ZSD110_MERCADEO['Pedidos'],Data_ZSD110_MERCADEO['Unidades'],Data_ZSD110_MERCADEO['Valor Neto']=Funtions.Clean_Num_List(Data_ZSD110_MERCADEO['Pedidos'],Data_ZSD110_MERCADEO['Unidades'],Data_ZSD110_MERCADEO['Valor Neto'])
# Cambio a valores Numericos Firme
Data_ZSD110_FIRME['Pedidos'],Data_ZSD110_FIRME['Unidades'],Data_ZSD110_FIRME['Valor Neto']=Funtions.Clean_Num_List(Data_ZSD110_FIRME['Pedidos'],Data_ZSD110_FIRME['Unidades'],Data_ZSD110_FIRME['Valor Neto'])
# Cambio a valores Numericos Pendiente Total
Data_ZSD110_PENDIENTES['Pedidos'],Data_ZSD110_PENDIENTES['Unidades'],Data_ZSD110_PENDIENTES['Valor Neto']=Funtions.Clean_Num_List(Data_ZSD110_PENDIENTES['Pedidos'],Data_ZSD110_PENDIENTES['Unidades'],Data_ZSD110_PENDIENTES['Valor Neto'])

In [63]:
#Aplicar consolidado de todas las tablas Mercadeo Firme Total
Consolidado_Mercadeo_Firme_Total=pd.merge(pd.merge(Data_ZSD110_MERCADEO,Data_ZSD110_FIRME,how="right",on="Can.distr."),Data_ZSD110_PENDIENTES,how="right",on="Can.distr.")
Consolidado_Mercadeo_Firme_Total=Consolidado_Mercadeo_Firme_Total.fillna(0)
Consolidado_Mercadeo_Firme_Total['Can.distr.']=Consolidado_Mercadeo_Firme_Total['Can.distr.'].replace({0:"TOTAL"})

In [64]:
Consolidado_Mercadeo_Firme_Total
Column_Consolidado_Mercadeo_Firme_Total=['Can.distr.', 'Pedidos M', 'Lineas M', 'Unidades M', 'Valor Neto M',
       'Pedidos F', 'Lineas F', 'Unidades F', 'Valor Neto F', 'Pedidos T',
       'Lineas T', 'Unidades T', 'Valor Neto T']

Consolidado_Clientes=Funtions.default_column(Column_Consolidado_Mercadeo_Firme_Total,Consolidado_Mercadeo_Firme_Total)

In [65]:
#Creo tabla total de la linea de clientes

Datos_lineas_clientes=pd.DataFrame([[1,"ECOMMERCE",0],[2,"Cencosud",0],[3,"Éxito Agenda",0],
              [4,"Canal 1",0],[5,"Canales 2,7,8",0],[6,"BAZAR",0],[7,"Mercadeo",0],
              [8,"Canales 14, 19 Citas",0],[9,"Extra agenda",0]],columns=["Prioridad","Clientes","Cantidad"])

In [66]:
#Linea Ecommerce
Datos_lineas_clientes['Clientes'][0]

'ECOMMERCE'

In [67]:
Consolidado_Clientes[Consolidado_Clientes["Can.distr."]=='ECOMMERCE']['Lineas F'].values[0] if 'ECOMMERCE' in Consolidado_Clientes["Can.distr."].to_list() else 0

47.0

In [68]:
Consolidado_Clientes[Consolidado_Clientes["Can.distr."]=='ECOMMERCE']['Lineas F'].values[0]

47.0

In [69]:
Consolidado_Clientes[Consolidado_Clientes["Can.distr."]=='ECOMMERCE']['Lineas F']

6    47.0
Name: Lineas F, dtype: float64

# Dash board

In [70]:
def Create_Table(Table,Name_id,fontSize='11px'):
    """
    - Table: Dataframe referencia a tabla
    - Name_id: Id de la tabla
    - Alias_libreria: Libreria Dash dash_table as dt
    """ 
    Tabla=dt.DataTable(
            id=Name_id,
            columns=[{"name": i, "id": i} for i in Table.columns],
            data=Table.to_dict("records"),
            style_data={
            'fontSize':fontSize
            },
            style_table={
                'margin': '0 auto',
                'border': '1px solid black',
                'borderCollapse': 'collapse'
            },
            style_header={
                'fontSize':fontSize,
                'backgroundColor': '#4074D5',
                'fontWeight': 'bold',
                'border': '1px solid black'
            },
            style_cell={
                'textAlign': 'center',
                'border': '1px solid black',
                'padding': '2px',
                'width': '15px'
            },
            )
    return(Tabla)

def Create_Dropdown(Column_Table,Estados,Name_id):
        Dropdown=dcc.Dropdown(
                        id=Name_id,
                        options=[{"label": st, "value": st} for st in Estados],
                        placeholder="-Select a State-",
                        multi=True,
                        value=Column_Table.unique())
        return(Dropdown)


def data_bars(df, column):
    n_bins = 100
    bounds = [i * (1.0 / n_bins) for i in range(n_bins + 1)]
    ranges = [
        ((df[column].sum() - df[column].min()) * i)
        for i in bounds
    ]
    styles = []
    
    for i in range(1, len(bounds)):
        min_bound = ranges[i - 1]
        max_bound = ranges[i]
        max_bound_percentage = bounds[i] * 100
        styles.append({
            'if': {
                'filter_query': (
                    '{{{column}}} >= {min_bound}' +
                    (' && {{{column}}} < {max_bound}' if (i < len(bounds) - 1) else '')
                ).format(column=column, min_bound=min_bound, max_bound=max_bound),
                'column_id': column
            },
            'background': (
                """
                    linear-gradient(90deg,
                    #2FCF7F 0%,
                    #2FCF7F {max_bound_percentage}%,
                    white {max_bound_percentage}%,
                    white 100%)
                """.format(max_bound_percentage=max_bound_percentage)
            ),
            'paddingBottom': 2,
            'paddingTop': 2
        })

    return styles

def Create_Table_Condition(df,Name_id,columns,fontSize='11px'):
    table=  dt.DataTable(id=Name_id,
            data=df.to_dict('records'),
            sort_action='native',
            columns=[{'name': i, 'id': i} for i in df.columns],
            style_data_conditional=(
                data_bars(df,columns)
            ),
            style_data={
                'fontSize':fontSize
            },
            style_table={
                'margin': '0 auto',
                'border': '1px solid black',
                'borderCollapse': 'collapse'
            },
            style_header={
                'fontSize':fontSize,
                'backgroundColor': '#4074D5',
                'fontWeight': 'bold',
                'border': '1px solid black'
            },
            style_cell={
                'textAlign': 'center',
                'border': '1px solid black',
                'padding': '5px',
                'width': '20px'
            },
            page_size=20
        )
    return(table)

In [71]:
Columns_Consolidado_Clientes=['Pedidos M', 'Lineas M', 'Unidades M', 'Valor Neto M',
       'Pedidos F', 'Lineas F', 'Unidades F', 'Valor Neto F', 'Pedidos T',
       'Lineas T', 'Unidades T', 'Valor Neto T']
Total_Consolidado_Clientes=Consolidado_Clientes[Consolidado_Clientes['Can.distr.']=='TOTAL']
Consolidado_Clientes=Consolidado_Clientes[Consolidado_Clientes['Can.distr.']!='TOTAL']

In [72]:
def Total_Consolidado(Consolidado_Clientes,Columns_Consolidado_Clientes,Total_Consolidado_Clientes):
    Diccionario_Total_Consolidado_Clientes=Total_Consolidado_Clientes.to_dict('records')[0]
    for i in Columns_Consolidado_Clientes:
        Diccionario_Total_Consolidado_Clientes[i]=Consolidado_Clientes[i].sum()
    Consolidado_Clientes=Consolidado_Clientes.to_dict("records")
    Consolidado_Clientes.append(Diccionario_Total_Consolidado_Clientes)
    return(Consolidado_Clientes)


In [73]:
def Create_figure(Dataframe,Column_X,list_column_y):
    Channels = Dataframe[Column_X].unique().tolist()

    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=Channels,
        y=Dataframe[list_column_y[0]].tolist(),
        name='Pedidos',
        marker_color='indianred'
    ))
        # Establecer el diseño del gráfico
    fig.add_trace(go.Bar(
        x=Channels,
        y=Dataframe[list_column_y[1]].tolist(),
        name='Lineas',
        marker_color='lightsalmon'
    ))
    fig.add_trace(go.Bar(
        x=Channels,
        y=Dataframe[list_column_y[2]].tolist(),
        name='Unidades',
        marker_color='blue'
    ))
    # Here we modify the tickangle of the xaxis, resulting in rotated labels.
    fig.update_layout(barmode='group', xaxis_tickangle=-45,margin=dict(l=0, r=0, t=30, b=0))
    return(fig)

In [74]:
Lineas_table_dinamica_Exito_entrega=sum(Tabla_Agenda_Exito['Cantidad'])

Lineas_table_dinamica_Cencosub_entrega=sum(Tabla_Agenda_Cencusub['Cantidad'])

data=np.array([["Exito",2,3,Lineas_table_dinamica_Exito_entrega],["Cencosub",4,5,Lineas_table_dinamica_Cencosub_entrega]])

df3=pd.DataFrame(data,index=["Exito","Cencosub"],columns=["Cliente","ZVMI","FIRME","Total"])
df3["Total"]=df3["Total"].astype(float)
        

app = Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP],suppress_callback_exceptions=True)
states_clase_pedido_Exito = Tabla_Agenda_Exito['Clase Pedido'].unique().tolist()
states_clase_pedido_Cencosub  =Tabla_Agenda_Cencusub['Clase Pedido'].unique().tolist()
states_clase_plataforma_Exito = Tabla_Agenda_Exito['Plataforma'].unique().tolist()
states_clase_plataforma_Cencosub=Tabla_Agenda_Cencusub['Plataforma'].unique().tolist()
states_Candistr=Consolidado_Clientes['Can.distr.'].unique().tolist()


Pague_1 = html.Div(children=[
    html.Div(className="row justify-content mb-3", children=[
        html.Div(className="col-md-3 "),
        html.Div(className="col-md-6 w-50", children=[
            html.Div(className="blockquote text-center", children=[
                html.H3("Consolidado"),
            ]),
            Create_Table_Condition(df3,"Consolidado_Agenda_Exito_Cencosub",'Total'),
        ]),
        html.Div(className="col-md-3 "),
    ]),
    html.Div(className="row justify-content", children=[
        html.Div(className="col-md-6 w-30 mx-auto mb-2", children=[
            html.Div(className="container bg-primary rounded-3 border border border-success p-2 mb-2 border-opacity-50 mb-2", children=[
                html.Div(className="row justify-content", children=[
                    html.Div(className="blockquote text-center text-white mb-2 ", children=[
                            html.H3("Agenda EXITO"),
                    ]),
                    html.Div(className="col-md-6 w-50 mx-auto mb-3", children=[
                        html.Label(['Clase Pedido'], style={'font-weight': 'bold', "text-align": "center"}),
                        Create_Dropdown(Tabla_Agenda_Exito['Clase Pedido'],states_clase_pedido_Exito,"filter_dropdown_Clase_Pedido_E"),  #Creamos Dropsown
                    ]),
                    html.Div(className="col-md-6 w-50 mx-auto mb-3", children=[
                        html.Label(['Plataforma'], style={'font-weight': 'bold', "text-align": "center"}),
                        Create_Dropdown(Tabla_Agenda_Exito['Plataforma'],states_clase_plataforma_Exito,"filter_dropdown_Plataforma_E"),   #Creamos Dropdown
                    ]),
                ]),                                   
            ]),
            Create_Table(Tabla_Agenda_Exito,"Tabla_Agenda_Exito"),    #Crear Tabla_Agenda_Exito   
        ]), 
        html.Div(className="col-md-6 w-50 mx-auto mb-2", children=[
            html.Div(className="container bg-primary rounded-3 border border-success p-2 mb-2 border-opacity-50 mb-2", children=[   
                html.Div(className="row justify-content", children=[  
                    html.Div(className="blockquote text-center text-white mb-2", children=[
                        html.H3("Agenda CENCOSUD"),
                    ]),
                    html.Div(className="col-md-6 w-50 mx-auto mb-3", children=[
                        html.Label(['Clase Pedido'], style={'font-weight': 'bold', "text-align": "start"}),
                        Create_Dropdown(Tabla_Agenda_Cencusub['Clase Pedido'],states_clase_pedido_Cencosub,"filter_dropdown_Clase_Pedido_CEN"),
                    ]),
                    html.Div(className="col-md-6 w-50 mx-auto", children=[
                        html.Label(['Plataforma'], style={'font-weight': 'bold', "text-align": "center"}),
                        Create_Dropdown(Tabla_Agenda_Cencusub['Plataforma'],states_clase_plataforma_Cencosub,"filter_dropdown_Plataforma_CEN"),   #Creamos Dropdown
                    ]),
                ]),
            ]),
            Create_Table(Tabla_Agenda_Cencusub,"Tabla entrega CENCOSUB"),   
        ]),
    ]),
])

Pague_2 = html.Div(children=[
    html.Div(className="row justify-content", children=[
        html.Div(className="col-md-8 w-30 p-2", children=[
                html.Div(className="blockquote text-center text-black mb-2 pe-5", children=[
                    html.H2("Total Pedidos"),
                    html.Label(['Plataforma'], style={'font-weight': 'bold', "text-align": "center"}),
                    Create_Dropdown(Consolidado_Clientes['Can.distr.'],states_Candistr,"filter_dropdown_Candistr"), 
                html.Div(className="row justify-content", children=[
                    Create_Table(Consolidado_Clientes,"Consolidado_Clientes","9px"),    #Crear Tabla_Agenda_Exito  
                   ]),
                html.Div(className="row justify-content mt-3", children=[
                    Create_Table(Datos_lineas_clientes,"Datos_lineas_clientes","9px"),    #Crear Tabla_Agenda_Exito 
                ]),
            ]),
        ]),
        html.Div(className="col-md-4 w-30 p-2", children=[
            html.Div(className="container bg-primary ms-3", children=[
                dcc.Graph(id='Grafico_Mercadeo',style={'width': '400px', 'height': '350px'},figure=Create_figure(Consolidado_Clientes,'Can.distr.',['Pedidos M','Lineas M','Unidades M'])),
                dcc.Graph(id='Grafico_Firme',style={'width': '400px', 'height': '350px'},figure=Create_figure(Consolidado_Clientes,'Can.distr.',['Pedidos F','Lineas F','Unidades F'])),
                dcc.Graph(id='Grafico_Total',style={'width': '400px', 'height': '350px'},figure=Create_figure(Consolidado_Clientes,'Can.distr.',['Pedidos T','Lineas T','Unidades T'])),
            ]),    
        ]),  
    ]),
])

Pague_3= html.Div(children=[
    html.H1('Prueba'),
])
app.layout = html.Div(children=[
    html.Div(className="container-fluid p-3 bg-primary text-white text-center", children=[
         html.H1("Pedidos Pendientes"),
    ]),

    dcc.Tabs(id="tabs-example-graph", value='tab-1-example-graph', children=[
        dcc.Tab(label='Tab One', value='tab-1-example-graph', children=[Pague_1]),
        dcc.Tab(label='Tab Two', value='tab-2-example-graph', children=[Pague_2]),
        dcc.Tab(label='Tab Tree', value='tab-3-example-graph', children=[Pague_3]),
    ]),
])



@app.callback(
    Output("Tabla_Agenda_Exito", "data"),
    Output("Tabla entrega CENCOSUB", "data"), 
    Output("Consolidado_Agenda_Exito_Cencosub", "data"), 
    Output("Consolidado_Clientes", "data"),
    Output("Grafico_Mercadeo", "figure"),
    Output('Grafico_Firme', "figure"),  
    Output('Grafico_Total', "figure"),
    Input("filter_dropdown_Clase_Pedido_E", "value"),
    Input("filter_dropdown_Plataforma_E", "value"),
    Input("filter_dropdown_Clase_Pedido_CEN", "value"),
    Input("filter_dropdown_Plataforma_CEN", "value"),
    Input("filter_dropdown_Candistr", "value"),
    Input('tabs-example-graph', 'value'),
)

def display_table(State_Clase_Pedido_EXT,State_Plataforma_EXT,State_Clase_Pedido_CEN,State_Plataforma_CEN,State_Candistr,tab):
    if tab == 'tab-1-example-graph':
        Output_Exito = Tabla_Agenda_Exito[Tabla_Agenda_Exito['Clase Pedido'].isin(State_Clase_Pedido_EXT) | Tabla_Agenda_Exito['Plataforma'].isin(State_Plataforma_EXT)]
        Output_Cencosub = Tabla_Agenda_Cencusub[Tabla_Agenda_Cencusub['Clase Pedido'].isin(State_Clase_Pedido_CEN)| Tabla_Agenda_Cencusub['Plataforma'].isin(State_Plataforma_CEN)]
        Output_Exito_ZVMI=Output_Exito[Output_Exito['Clase Pedido']=='ZVMI']
        Output_Cencosub_ZVMI=Output_Cencosub[Output_Cencosub['Clase Pedido']=='ZVMI']
        Output_Exito_ZED1_ZEDI=Output_Exito[(Output_Exito['Clase Pedido']=='ZED1') | (Output_Exito['Clase Pedido']=='ZEDI')] # | Output_Exito['Clase Pedido']=='ZEDI'
        Output_Cencosub_ZED1_ZEDI=Output_Cencosub[(Output_Cencosub['Clase Pedido']=='ZED1') | (Output_Cencosub['Clase Pedido']=='ZEDI')] # | Output_Cencosub['Clase Pedido']=='ZEDI'
        df3["ZVMI"][0]=Output_Exito_ZVMI["Cantidad"].sum()
        df3["ZVMI"][1]=Output_Cencosub_ZVMI["Cantidad"].sum()
        df3["FIRME"][0]=Output_Exito_ZED1_ZEDI["Cantidad"].sum()
        df3["FIRME"][1]=Output_Cencosub_ZED1_ZEDI["Cantidad"].sum()
        df3["Total"]=df3["ZVMI"]+df3["FIRME"]
        dff3=df3
        return Output_Exito.to_dict("records"),Output_Cencosub.to_dict("records"),dff3.to_dict("records"),dash.no_update,dash.no_update,dash.no_update,dash.no_update
    elif tab == 'tab-2-example-graph':
        Output_Consolidado=Consolidado_Clientes[Consolidado_Clientes['Can.distr.'].isin(State_Candistr)]
        Output_Consolidado_figure=Consolidado_Clientes[Consolidado_Clientes['Can.distr.'].isin(State_Candistr)]
        figure_mercadeo=Create_figure(Output_Consolidado_figure,'Can.distr.',['Pedidos M','Lineas M','Unidades M'])
        figure_firme=Create_figure(Output_Consolidado_figure,'Can.distr.',['Pedidos F','Lineas F','Unidades F'])
        figure_total=Create_figure(Output_Consolidado_figure,'Can.distr.',['Pedidos T','Lineas T','Unidades T'])
        Output_Consolidado=Total_Consolidado(Output_Consolidado,Columns_Consolidado_Clientes,Total_Consolidado_Clientes)
        return dash.no_update,dash.no_update,dash.no_update,Output_Consolidado,figure_mercadeo,figure_firme,figure_total
    elif tab == 'tab-3-example-graph':
        return(dash.no_update,dash.no_update,dash.no_update,dash.no_update,dash.no_update,dash.no_update,dash.no_update)


app.run(host='0.0.0.0', port=8000, debug=False)

Dash is running on http://0.0.0.0:8000/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.15.1.68:8000
Press CTRL+C to quit


127.0.0.1 - - [07/Jul/2023 11:20:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2023 11:20:18] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2023 11:20:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2023 11:20:19] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Jul/2023 11:20:19] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1C:\Users\prac.ingindustrial2\AppData\Local\Temp\ipykernel_12104\442972296.py:136: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\prac.ingindustrial2\AppData\Local\Temp\ipykernel_12104\442972296.py:137: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

# Primer parte con reporte de procesos diarios ZSD10 y ZD79


In [75]:
Reporte_Procesos_Diarios=pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\Archivos_CSV\\Tabla_Consolidado_Diaria.txt",delimiter='\t')

In [76]:
Agenda=["85","20","146","149","50","138","45"]
Reporte_Procesos_Diarios['Nº Pedido cliente']=Funtions.complete_pedidos(Reporte_Procesos_Diarios['Nº Pedido cliente'],Agenda)

In [77]:
variant="PEDIDOS"

In [78]:
transsaccion_ZSD10="ZSD10"
provision_ZSD10="PEDIDOS"
variant_ZSD10="JESPINOSAP"




def Search_ZSD10(transsaccion,Series,variant,provision,session):
    """
    Transsaccion: Transsacion a buscar
    Series: Columna del dataframe que quiero copiar
    session: session del usuario
    variant: Variante a buscar
    provision: disposicion de interes
    session: session activa
    """
    session.StartTransaction(transsaccion)
    session.findById("wnd[0]/tbar[1]/btn[17]").press()
    session.findById("wnd[1]/usr/txtV-LOW").text = provision
    session.findById("wnd[1]/usr/txtENAME-LOW").text = variant
    session.findById("wnd[1]/usr/txtV-LOW").caretPosition = 8
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    Series=Series.to_clipboard(index=False, header=False)
    session.findById("wnd[0]/usr/btn%_SP$00013_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    try:
        tabla=session.findById("wnd[0]/usr/cntlCONTAINER/shellcont/shell")
        return(tabla)
    except:
        return(session)

In [79]:
Search_ZSD10(transsaccion_ZSD10,Reporte_Procesos_Diarios['Nº Pedido cliente'],variant_ZSD10,provision_ZSD10,session)

<COMObject <unknown>>

# Puedo omitir este paso y con otro posterior

In [80]:
Ruta_ZSD10_Total_Exito="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
Name_ZSD10_Total_Pedidos="Lineas_Exto"
SAP_GUI.Export_TXT2(Name_ZSD10_Total_Pedidos,session,Ruta_ZSD10_Total_Exito)

In [81]:
Pedidos_Dia_ZSD10=pd.read_csv(Ruta_ZSD10_Total_Exito+Name_ZSD10_Total_Pedidos+'.txt',skiprows=5,delimiter='\t')
Pedidos_Dia_ZSD10=Funtions.Clean_Columns(Pedidos_Dia_ZSD10)
Pedidos_Dia_ZSD10=Funtions.default_column(Columns_Data_ZSD10,Pedidos_Dia_ZSD10)
Pedidos_Dia_ZSD10=Pedidos_Dia_ZSD10[Pedidos_Dia_ZSD10["Doc.venta"].notnull()]    #Elimino espacios vacios 
Pedidos_Dia_ZSD10["Doc.venta"]=Funtions.Clean_int_to_str(Pedidos_Dia_ZSD10["Doc.venta"])   #Combierto columna

In [82]:
transsaccion_ZSD76="ZSD76"
provision_ZSD76="PEDIDOS"
variant_ZSD76="JESPINOSAP"

In [92]:
def Search_ZSD76(transsaccion,Series,session):
    """
    Transsaccion: Transsacion a buscar
    Series: Columna del dataframe que quiero copiar
    session: session del usuario
    """
    session.StartTransaction(transsaccion)
    Series=Series.to_clipboard(index=False, header=False)
    session.findById("wnd[0]/usr/btn%_SP$00010_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    try:
        tabla=session.findById("wnd[0]/usr/cntlCONTAINER/shellcont/shell")
        return(tabla)
    except:
        return(session)

In [94]:
tabla=Search_ZSD76(transsaccion_ZSD76,Pedidos_Dia_ZSD10["Doc.venta"].drop_duplicates(),session)

In [95]:
Pedidos_Dia_ZSD10["Doc.venta"]

0    903148603
Name: Doc.venta, dtype: object

In [85]:
Ruta_ZSD10_Detalle_Entregas="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
Name_ZSD10_Detalle_Entregas="Detalle_Entregas"
#SAP_GUI.Export_TXT2(Name_ZSD10_Detalle_Entregas,session,Ruta_ZSD10_Detalle_Entregas)
SAP_GUI.Export_TXT(Name_ZSD10_Detalle_Entregas,session,Ruta_ZSD10_Detalle_Entregas)

com_error: (-2147352567, 'Ocurrió una excepción.', (619, 'SAP Frontend Server', 'The control could not be found by id.', 'C:\\Program Files (x86)\\SAP\\FrontEnd\\SAPgui\\sapfront.HLP', 393215, 0), None)

In [ ]:
Detalle_Entregas=pd.read_csv(Ruta_ZSD10_Detalle_Entregas+Name_ZSD10_Detalle_Entregas+".txt",skiprows=5,delimiter='\t')
Detalle_Entregas=Funtions.Clean_Columns(Detalle_Entregas)
Detalle_Entregas=Funtions.default_column(Column_Data_ZSD76,Detalle_Entregas)
Detalle_Entregas=Detalle_Entregas[Detalle_Entregas["Solic."].notnull()]

# Tener presente para lineas total exito

In [ ]:
Lineas_Total_Exito=Detalle_Entregas.groupby("Cantidad entrega").count()["Solic."].sum()

In [ ]:
Lineas_Total_Exito

# Reporte de materias

In [ ]:
transsaccion="ZSD035D"
variant="JESPINOSAP"
provision="INFORME FACTU"

In [ ]:
Un_dia_atras=(datetime.now()-timedelta(days=1)).date()
Un_dia_atras=Funtions.stand_day(Un_dia_atras)
SAP_GUI.Search_ZSD035D(transsaccion,session,provision,Un_dia_atras,variant)
# Pendiente Firme
Name_ZSD035D="Pendiente_Clientes"  # Revisar suele no guardar
SAP_GUI.Export_TXT2(Name_ZSD035D,session)

In [ ]:
# Pendiente Firme
url_ZSD035D="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\Archivos_CSV\\"+str(Name_ZSD035D)+".txt"
Data_ZSD035D=pd.read_csv(url_ZSD035D,skiprows=5,delimiter="\t")
Data_ZSD035D=Funtions.Clean_Columns(Data_ZSD035D)
Data_ZSD035D=Data_ZSD035D[Data_ZSD035D['Material'].notnull()]
#Data_ZSD035D["Lineas"]=Data_ZSD035D["Lineas"].apply(lambda x:Funtions.Clean_num(x))    
#Tener encuenta esta lineas puede este en dato numerico

In [ ]:
#Limpio datos de la columnas a numeros

Data_ZSD035D['Cantidad'], Data_ZSD035D['Valor Neto']= Funtions.Clean_Num_List(Data_ZSD035D['Cantidad'],Data_ZSD035D['Valor Neto'])

In [ ]:
#Canal01_Entregado_Exluido.pivot_table(,,['Material', 'Valor Neto'])

Table_Data_ZSD035D=Data_ZSD035D.pivot_table(index=['CDis'],aggfunc={'Material':'count','Pedido':'count','Valor Neto':sum,'Cantidad':sum}).reset_index()[['CDis','Pedido', 'Material', 'Cantidad', 'Valor Neto']]
Table_Data_ZSD035D=Funtions.Clean_Columns(Table_Data_ZSD035D)

Table_Data_ZSD035D

In [ ]:
# Debo de filtrar por ZVMI y exito y cencusub del día anterior  

"""Tendria que usar el excel de sharepoint como sugerencia"""

filtro_ZVMI_Exito=Tabla_Agenda_Exito["Clase Pedido"]=="ZVMI"
filtro_ZVMI_Cencosub=Tabla_Agenda_Exito["Clase Pedido"]=="ZVMI"

Tabla_Agenda_Exito_ZVMI=Tabla_Agenda_Exito.loc[filtro_ZVMI_Exito]
Tabla_Agenda_Cencosub_ZVMI=Tabla_Agenda_Cencusub.loc[filtro_ZVMI_Cencosub]

In [ ]:
Tabla_Agenda_Cencosub_ZVMI

In [ ]:
Lineas_Tabla_Agenda_Exito_ZVMI=Tabla_Agenda_Exito_ZVMI["Cantidad"].sum()
Lineas_Tabla_Agenda_Cencosub_ZVMI=Tabla_Agenda_Cencosub_ZVMI["Cantidad"].sum()
Unidades_Tabla_Agenda_Exito_ZVMI=Tabla_Agenda_Exito_ZVMI.Material.sum()
Unidades_Tabla_Agenda_Cencosub_ZVMI=Tabla_Agenda_Cencosub_ZVMI.Material.sum()
Valor_neto_Tabla_Agenda_Exito_ZVMI=Tabla_Agenda_Exito_ZVMI["Valor Neto"].sum()
Valor_neto_Tabla_Agenda_Cencosub_ZVMI=Tabla_Agenda_Cencosub_ZVMI["Valor Neto"].sum()

In [ ]:
# ZSD110 FIRME
url_ZSD037_Pedidos_Exito_Cencosub="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_ZSD037)+".txt"

Data_ZSD110_Pedidos_Exito_Cencosub=pd.read_csv(url_ZSD037_Pedidos_Exito_Cencosub,skiprows=5,delimiter="\t")
Data_ZSD110_Pedidos_Exito_Cencosub=Funtions.Clean_Columns(Data_ZSD110_Pedidos_Exito_Cencosub)
Data_ZSD110_Pedidos_Exito_Cencosub=Funtions.default_column(default_column_Pedidos,Data_ZSD110_Pedidos_Exito_Cencosub)
Data_ZSD110_Pedidos_Exito_Cencosub=Data_ZSD110_Pedidos_Exito_Cencosub[Data_ZSD110_Pedidos_Exito_Cencosub['Nº pedido cliente'].notnull()]

In [ ]:
Tabla_Agenda_Exito_ZVMI["Nº pedido cliente"].to_clipboard(index=False, header=False)

In [ ]:
Tabla_Agenda_Exito_Cencosub_ZVMI=pd.concat([Tabla_Agenda_Exito_ZVMI["Nº pedido cliente"],Tabla_Agenda_Cencosub_ZVMI["Nº pedido cliente"]])

In [ ]:
Tabla_Agenda_Exito_Cencosub_ZVMI

In [ ]:
transsaccion_ZSD10="ZSD10"
provision_ZSD10="PEDIDOS"
variant_ZSD10="JESPINOSAP"


def Search_ZSD10(transsaccion,Series,variant,provision,session):
    """
    Transsaccion: Transsacion a buscar
    Series: Columna del dataframe que quiero copiar
    session: session del usuario
    variant: Variante a buscar
    provision: disposicion de interes
    session: session activa
    """
    session.StartTransaction(transsaccion)
    session.findById("wnd[0]/tbar[1]/btn[17]").press()
    session.findById("wnd[1]/usr/txtV-LOW").text = provision
    session.findById("wnd[1]/usr/txtENAME-LOW").text = variant
    session.findById("wnd[1]/usr/txtV-LOW").caretPosition = 8
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    Series=Series.to_clipboard(index=False, header=False)
    session.findById("wnd[0]/usr/btn%_SP$00013_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    try:
        tabla=session.findById("wnd[0]/usr/cntlCONTAINER/shellcont/shell")
        return(tabla)
    except:
        return(session)


Search_ZSD10(transsaccion_ZSD10,Tabla_Agenda_Exito_Cencosub_ZVMI,variant_ZSD10,provision_ZSD10,session)
Name_ZSD010_Exito_Cencosub_ZVMI="Exito_Cencosub_ZVMI"
Ruta_Exito_ZSD010_ZVMI="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_ZSD010_Exito_Cencosub_ZVMI,session,Ruta_Exito_ZSD010_ZVMI)


In [ ]:
# Exportar de la transsaccion ZSD10 exporta los archivos

url_ZSD10_Exito_Cencosub_ZVMI=Ruta_Exito_ZSD010_ZVMI+Name_ZSD010_Exito_Cencosub_ZVMI+".txt"
Data_ZSD10_Exito_Cencosub_ZVMI=pd.read_csv(url_ZSD10_Exito_Cencosub_ZVMI,skiprows=5,delimiter="\t")
Data_ZSD10_Exito_Cencosub_ZVMI=Funtions.Clean_Columns(Data_ZSD10_Exito_Cencosub_ZVMI)
Data_ZSD10_Exito_Cencosub_ZVMI=Funtions.default_column(Columns_Data_ZSD10_Exito_Cencosub_ZVMI,Data_ZSD10_Exito_Cencosub_ZVMI)
Data_ZSD10_Exito_Cencosub_ZVMI=Data_ZSD10_Exito_Cencosub_ZVMI[Data_ZSD10_Exito_Cencosub_ZVMI["Cliente"].notnull()]

# Transformo datos 
Data_ZSD10_Exito_Cencosub_ZVMI["Unidades"],Data_ZSD10_Exito_Cencosub_ZVMI["ValorNeto"],Data_ZSD10_Exito_Cencosub_ZVMI["Total Pedidos"]=Funtions.Clean_Num_List(Data_ZSD10_Exito_Cencosub_ZVMI["Unidades"],Data_ZSD10_Exito_Cencosub_ZVMI["ValorNeto"],Data_ZSD10_Exito_Cencosub_ZVMI["Total Pedidos"])
# Tranformo la columna que tiene .0 por si es necesario
try:
    Data_ZSD10_Exito_Cencosub_ZVMI['Doc.venta']=Funtions.Clean_int_to_str(Data_ZSD10_Exito_Cencosub_ZVMI['Doc.venta'])
except:
    None

In [ ]:
Datos_Agenda_Exito_ZVMI=list(Tabla_Agenda_Exito_ZVMI["Nº pedido cliente"].unique())    #Utilizo los valores de la tabla ya que con estos pedidos del Exito busco en la ZSD10
Datos_Agenda_Cencosub_ZVMI=list(Tabla_Agenda_Cencosub_ZVMI["Nº pedido cliente"].unique()) #Utilizo los valores de la tabla ya que con estos pedidos de Cencosub busco en la ZSD10

In [ ]:
Data_ZSD10_Exito_ZVMI=Data_ZSD10_Exito_Cencosub_ZVMI.loc[Data_ZSD10_Exito_Cencosub_ZVMI["Nº pedido cliente"].isin(Datos_Agenda_Exito_ZVMI)]    
Data_ZSD10_Cencosub_ZVMI=Data_ZSD10_Exito_Cencosub_ZVMI.loc[Data_ZSD10_Exito_Cencosub_ZVMI["Nº pedido cliente"].isin(Datos_Agenda_Cencosub_ZVMI)]         #in list(Datos_Agenda_Exito_ZVMI)

In [ ]:
#Resumenes numericos de tablas Cencosub

Resumen_Exito_ZVMI=Data_ZSD10_Exito_ZVMI.pivot_table(index=['Material'],aggfunc={'Nº pedido cliente':'count','ValorNeto':sum,'Unidades':sum,'Total Pedidos':sum}).reset_index()
Resumen_Cencosub_ZVMI=Data_ZSD10_Cencosub_ZVMI.pivot_table(index=['Material'],aggfunc={'Nº pedido cliente':'count','ValorNeto':sum,'Unidades':sum,'Total Pedidos':sum}).reset_index()

In [ ]:
Lineas_Tabla_Agenda_Exito_ZVMI=Resumen_Exito_ZVMI['Nº pedido cliente'].sum()
Lineas_Tabla_Agenda_Cencosub_ZVMI=Resumen_Cencosub_ZVMI['Nº pedido cliente'].sum()
Unidades_Tabla_Agenda_Exito_ZVMI=Resumen_Exito_ZVMI.Unidades.sum()
Unidades_Tabla_Agenda_Cencosub_ZVMI=Resumen_Cencosub_ZVMI.Unidades.sum()
Valor_neto_Tabla_Agenda_Exito_ZVMI=Resumen_Exito_ZVMI['ValorNeto'].sum()
Valor_neto_Tabla_Agenda_Cencosub_ZVMI=Resumen_Cencosub_ZVMI['ValorNeto'].sum()
Total_Pedidos_Agenda_Exito_ZVMI=Resumen_Exito_ZVMI['Total Pedidos'].sum()
Total_Pedidos_Agenda_Cencosub_ZVMI=Resumen_Cencosub_ZVMI['Total Pedidos'].sum()

In [ ]:
#comit